In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-encoding': 'gzip, deflate, br, zstd',
    'connection': 'keep-alive',
    'host': 'austlii.edu.au',
    'referer': 'https://austlii.edu.au/',
    'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'
}
URL = "http://www.lawcite.org"

In [ ]:
# Set up Chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument(f'user-agent={headers["user-agent"]}')

driver = webdriver.Chrome(options=chrome_options)
driver.get(URL)
driver.implicitly_wait(0)

# Search for "Hong Kong" in the juris field
juris_input = driver.find_element(By.ID, "juris")
juris_input.send_keys("Hong Kong")
juris_input.submit()

# Allow large page loads
if len(driver.find_elements(By.CLASS_NAME, "message-box")) > 0:
    continue_button = driver.find_element(By.CSS_SELECTOR,"input[type='submit'][value='Continue']")
    continue_button.click()

wait = WebDriverWait(driver, 20)

# Ensure results are present (adjust selector to your table/container)
results_rows = wait.until(EC.presence_of_all_elements_located(
    (By.CSS_SELECTOR, "table tr")))  # refine selector if you can
first_row = results_rows[0] if results_rows else None

# Click “Sort by Citation Frequency”
sort_link = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, "a[title='Sort by Citation Frequency']")))
try:
    sort_link.click()
except Exception:
    driver.execute_script("arguments[0].click();", sort_link)

# Sort by Citation Index
sort_button = driver.find_element(By.CSS_SELECTOR, "a[title='Sort by Citation Frequency']")
sort_button.click()

driver.page_source  # This will load the page content
with open("lawcite.html", "w", encoding="utf-8") as file:
    file.write(driver.page_source)

In [ ]:
import pandas as pd
import os
import requests
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
df = pd.read_html("lawcite.html", header=0)[1]
df.head(2)

,Case Name,Citation(s),Court,Jurisdiction,Date,Full Text,Unnamed: 6,Citation Index †
0,Chan Kam Nga v Director of Immigration,[1999] HKCFA 16; [1999] 1 HKC 347; (1999) 2 HK...,Hong Kong Court of Final Appeal,Hong Kong,29 Jan 1999,HKLII,NaN,5051.0
1,Nupur Mst v Director of Immigration,[2018] HKCA 524,Hong Kong Court of Appeal,Hong Kong,16 Aug 2018,HKLII,NaN,3994.0


In [ ]:
df = pd.read_html("lawcite.html", header=0)[1]
df.columns = df.columns.str.strip()
df = df.dropna(axis=1, how='all')  # Drop columns that are completely empty
df.columns = df.columns.str.replace('†', '', regex=False)  # Remove '†' from column names
df.columns = df.columns.str.strip() # Clean up column names
print(df.columns)  # Check column names after cleaning

df["Citation Index"] = df["Citation Index"].astype(float)  # Ensure 'Citation Index' is treated as string
df.sort_values(by="Citation Index", inplace=True, ascending=False)  # Sort by 'Citation Index'
df.to_csv("law_cite_data.csv", index=False)
df.head()

Index(['Case Name', 'Citation(s)', 'Court', 'Jurisdiction', 'Date',
       'Full Text', 'Citation Index'],
      dtype='object')


,Case Name,Citation(s),Court,Jurisdiction,Date,Full Text,Citation Index
0,Chan Kam Nga v Director of Immigration,[1999] HKCFA 16; [1999] 1 HKC 347; (1999) 2 HK...,Hong Kong Court of Final Appeal,Hong Kong,29 Jan 1999,HKLII,5051.0
1,Nupur Mst v Director of Immigration,[2018] HKCA 524,Hong Kong Court of Appeal,Hong Kong,16 Aug 2018,HKLII,3994.0
2,TK v Jenkins,[2012] HKCA 502; [2013] 1 HKC 526,Hong Kong Court of Appeal,Hong Kong,21 Nov 2012,HKLII,3110.0
3,Re Lakhwinder Singh,[2018] HKCA 246,Hong Kong Court of Appeal,Hong Kong,1 Jun 2018,HKLII,1921.0
4,Re Daljit Singh,[2018] HKCA 328,Hong Kong Court of Appeal,Hong Kong,17 Jul 2018,HKLII,1880.0


In [ ]:
len(df[df["Citation Index"] >= 100])

In [ ]:
len(df[df["Citation Index"] >= 50])

In [ ]:
len(df[df["Citation Index"] >= 10])

In [ ]:
len(df[df["Citation Index"] >= 1])

In [ ]:
len(df[df["Citation Index"].isna()])  # Count non-null entries in 'Citation Index'